# Pre procesamiento de Datos

El pre procesamiento de los dato es un paso clave en el Machine/Deep learning, pues en la mayoria de los casos los datasets de entrenamiento no vienen listos o formateados lo suficiente como para ser usados directamente como entrenamiento, por lo mismo se realiza este proceso.

En este notebook se desarrolla el pre procesamiento de los datos de entrenamiento para el proyecto final

En este notebook se han realizado diversas transformaciones y preprocesamientos sobre los datos, tales como la imputación de valores nulos, la normalización de variables, la codificación de variables categóricas y ordinales, y la preparación de los conjuntos de datos para su uso en modelos de Machine Learning. Además, se han implementado técnicas como el encoding suavizado y el one-hot encoding para el tratamiento de variables categóricas.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [2]:
# Cargamos el conjunto de datos
df = pd.read_csv('data/train.csv')

In [3]:
# Observamos las primeras filas del DataFrame
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [5]:
# Se eliminala columna 'ID' ya que no aporta información relevante para el análisis
df.drop(columns=['ID'], inplace=True)

NOTA: Este analisis de Chi cuadrado deberia haberse realizado en el archivo anterior, sin embargo fue pospuesto por tiempo

In [7]:
from scipy.stats import chi2_contingency

Las siguientes ocho celdas realizan diversas transformaciones y análisis sobre los datos. Primero, se evalúa la independencia entre las variables categóricas E_PRGM_ACADEMICO y E_PRGM_DEPARTAMENTO con respecto a RENDIMIENTO_GLOBAL, utilizando tablas de contingencia y la prueba de Chi-cuadrado para calcular el valor p y determinar si existe una relación significativa. También se calcula el número de categorías únicas en estas variables. Posteriormente, se transforma la variable `RENDIMIENTO_GLOBAL` en una representación ordinal numérica (`RENDIMIENTO_GLOBAL_NUM`) para facilitar su uso en análisis posteriores. Además, se implementa un encoding suavizado para las variables `E_PRGM_ACADEMICO` y `E_PRGM_DEPARTAMENTO`, calculando una combinación ponderada entre la media global y la media por grupo, asignando estos valores a nuevas columnas y eliminando las columnas originales.

Finalmente, se crean y guardan los mapas de encoding generados para las variables categóricas, permitiendo su reutilización en futuros análisis. Estas transformaciones buscan preparar los datos para su uso en modelos de Machine Learning, asegurando que las variables categóricas sean representadas de manera adecuada y que las relaciones entre las variables sean analizadas y comprendidas.

In [ ]:
# Análisis de independencia entre 'E_PRGM_DEPARTAMENTO' y 'RENDIMIENTO_GLOBAL'
cont = pd.crosstab(df["E_PRGM_ACADEMICO"], df["RENDIMIENTO_GLOBAL"])
chi2, p, dof, exp = chi2_contingency(cont)
print(f"Chi-cuadrado p-value: {p:.8f} {'(Significativo)' if p < 0.05 else ''}")

Chi-cuadrado p-value: 0.00000000 (Significativo)


In [ ]:
# Análisis de independencia entre 'E_PRGM_DEPARTAMENTO' y 'RENDIMIENTO_GLOBAL'
cont = pd.crosstab(df["E_PRGM_DEPARTAMENTO"], df["RENDIMIENTO_GLOBAL"])
chi2, p, dof, exp = chi2_contingency(cont)
print(f"Chi-cuadrado p-value: {p:.8f} {'(Significativo)' if p < 0.05 else ''}")

Chi-cuadrado p-value: 0.00000000 (Significativo)


In [11]:
# Numero de valores unicos de "E_PRGM_ACADEMICO" y de "E_PRGM_DEPARTAMENTO"
df["E_PRGM_ACADEMICO"].unique().size, df["E_PRGM_DEPARTAMENTO"].unique().size

(948, 31)

In [12]:
map_ordinal = {
    'bajo': 1,
    'medio-bajo': 2,
    'medio-alto': 3,
    'alto': 4
}
df['RENDIMIENTO_GLOBAL_NUM'] = df['RENDIMIENTO_GLOBAL'].map(map_ordinal)


In [ ]:
# Media global del target
global_mean = df['RENDIMIENTO_GLOBAL_NUM'].mean()

# Conteo y media por programa
agg_prg = df.groupby("E_PRGM_ACADEMICO")['RENDIMIENTO_GLOBAL_NUM'].agg(['mean', 'count'])
agg_prg.columns = ['mean_target', 'count']

# Parámetro de suavizado (ajústalo según la dispersión de tus datos)
k = 20  

# Fórmula del smoothing:
# encoding = (mean_target * count + global_mean * k) / (count + k)
agg_prg['encoded'] = (agg_prg['mean_target'] * agg_prg['count'] + global_mean * k) / (agg_prg['count'] + k)

# --- 3️⃣ Asignar el encoding al dataframe original ---
df['E_PRGM_ACADEMICO_ENC'] = df['E_PRGM_ACADEMICO'].map(agg_prg['encoded'])

# --- 4️⃣ Rellenar posibles valores faltantes con la media global ---
df['E_PRGM_ACADEMICO_ENC'].fillna(global_mean, inplace=True)


C:\Users\JUANJO\AppData\Local\Temp\ipykernel_12244\436551978.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['E_PRGM_ACADEMICO_ENC'].fillna(global_mean, inplace=True)


In [18]:
df.drop(columns=['E_PRGM_ACADEMICO'], inplace=True)

In [ ]:
# Media global del target
global_mean = df['RENDIMIENTO_GLOBAL_NUM'].mean()

# Conteo y media por departamento
agg_dep = df.groupby("E_PRGM_DEPARTAMENTO")['RENDIMIENTO_GLOBAL_NUM'].agg(['mean', 'count'])
agg_dep.columns = ['mean_target', 'count']

# Parámetro de suavizado (ajústalo según la dispersión de tus datos)
k = 20  

# Fórmula del smoothing:
# encoding = (mean_target * count + global_mean * k) / (count + k)
agg_dep['encoded'] = (agg_dep['mean_target'] * agg_dep['count'] + global_mean * k) / (agg_dep['count'] + k)

# --- 3️⃣ Asignar el encoding al dataframe original ---
df['E_PRGM_DEPARTAMENTO_ENC'] = df['E_PRGM_DEPARTAMENTO'].map(agg_dep['encoded'])

# --- 4️⃣ Rellenar posibles valores faltantes con la media global ---
df['E_PRGM_DEPARTAMENTO_ENC'].fillna(global_mean, inplace=True)
df.drop(columns=['E_PRGM_DEPARTAMENTO'], inplace=True)

C:\Users\JUANJO\AppData\Local\Temp\ipykernel_12244\2465231645.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['E_PRGM_DEPARTAMENTO_ENC'].fillna(global_mean, inplace=True)


In [ ]:
# Finalizamos creando los mapas de encoding para futuras referencias
encoding_map_prg = agg_prg['encoded'].to_dict()
encoding_map_dep = agg_dep['encoded'].to_dict()


### Codificación de variables binarias

En esta celda, se identifican y transforman las columnas binarias del DataFrame `df`. Una columna binaria es aquella que tiene exactamente dos valores únicos (excluyendo valores nulos). El objetivo es convertir estas columnas en valores numéricos (0 y 1) para facilitar su uso en modelos de Machine Learning.

1. **Identificación de columnas binarias**: 
    - Se recorre cada columna del DataFrame `df` y se cuenta el número de valores únicos, excluyendo los valores nulos.
    - Si una columna tiene exactamente dos valores únicos, se considera binaria.

2. **Transformación de columnas binarias**:
    - Si la columna contiene valores booleanos (`True` y `False`), se convierten directamente a 1 y 0, respectivamente.
    - Para columnas no booleanas, se asigna 0 al primer valor único encontrado y 1 al segundo valor único.

3. **Mantenimiento de valores nulos**:
    - Las columnas transformadas se convierten al tipo `Int64`, que permite preservar valores nulos (`NaN`).

4. **Almacenamiento de mapeos**:
    - Se guarda un diccionario `binary_maps` que contiene los mapeos utilizados para cada columna binaria. Esto permite rastrear cómo se realizó la transformación.

5. **Resumen de la transformación**:
    - Se imprime el número total de columnas binarias transformadas y los mapeos utilizados para cada una.

Esta transformación es útil para estandarizar las variables binarias y prepararlas para su uso en algoritmos que requieren datos numéricos.

In [25]:
from pandas.api.types import is_bool_dtype

binary_maps = {}  # guardará mapping por columna -> {valor_original: 0/1}

for col in df.columns:
    # contar únicos excluyendo NaN
    n_uniques = df[col].dropna().nunique()
    if n_uniques == 2:
        # valores en orden de aparición (determinista según el dataframe)
        uniques = list(pd.Series(df[col].dropna().unique()))
        
        # caso booleano explícito: True->1, False->0
        if is_bool_dtype(df[col]) or set(map(type, uniques)) == {bool}:
            df[col] = df[col].astype('Int64')  # True->1, False->0, NaN preservado
            binary_maps[col] = {True: 1, False: 0}
        else:
            # crear mapping: primer valor -> 0, segundo valor -> 1
            mapping = {uniques[0]: 0, uniques[1]: 1}
            # mapear dejando NaN intactos y usar Int64 para permitir NA
            df[col] = df[col].map(mapping).astype('Int64')
            binary_maps[col] = mapping

# Mostrar resumen de columnas convertidas y sus mapeos
print(f"Columnas binarias convertidas: {len(binary_maps)}")
for c, m in binary_maps.items():
    print(f" - {c}: {m}")

Columnas binarias convertidas: 7
 - F_TIENEINTERNET: {'Si': 0, 'No': 1}
 - F_TIENELAVADORA: {'Si': 0, 'No': 1}
 - F_TIENEAUTOMOVIL: {'Si': 0, 'No': 1}
 - E_PRIVADO_LIBERTAD: {'N': 0, 'S': 1}
 - E_PAGOMATRICULAPROPIO: {'No': 0, 'Si': 1}
 - F_TIENECOMPUTADOR: {'Si': 0, 'No': 1}
 - F_TIENEINTERNET.1: {'Si': 0, 'No': 1}


Se eliminan las siguientes celdas:

1. `E_PRIVADO_LIBERTAD` demostró no ser significativa en el archivo exploracion y analisis
2. `F_TIENEINTERNET.1` está repetida, pues está en `F_TIENEINTERNET`

In [ ]:
df.drop(columns=['E_PRIVADO_LIBERTAD', 'F_TIENEINTERNET.1'], inplace=True)


### Explicación de las celdas 20 a 30

En las celdas 20 a 30 se realizan transformaciones y codificaciones sobre diversas variables categóricas y ordinales del DataFrame `df`. Estas transformaciones tienen como objetivo preparar los datos para su uso en modelos de Machine Learning, asegurando que las variables sean representadas de manera adecuada y que se mantenga la integridad de los datos.

Primero, se exploran las variables `E_VALORMATRICULAUNIVERSIDAD`, `E_HORASSEMANATRABAJA`, y `F_EDUCACIONPADRE`, identificando sus valores únicos y mapeándolos a valores numéricos ordinales mediante diccionarios de mapeo. Posteriormente, se aplica el mismo proceso a las variables `F_EDUCACIONMADRE` y `F_ESTRATOVIVIENDA`, utilizando el mismo diccionario de mapeo para las variables relacionadas con la educación. Finalmente, se eliminan columnas redundantes o innecesarias, como `F_EDUCACIONPADRE_ORD`, y se realiza una última transformación sobre la variable `F_ESTRATOVIVIENDA`, asignando valores ordinales y preservando valores nulos (`NA`) cuando sea necesario. Estas transformaciones aseguran que las variables categóricas y ordinales sean representadas de manera consistente y numérica, facilitando su uso en modelos predictivos.

In [ ]:
df["E_VALORMATRICULAUNIVERSIDAD"].unique()

array(['Entre 5.5 millones y menos de 7 millones',
       'Entre 2.5 millones y menos de 4 millones',
       'Entre 4 millones y menos de 5.5 millones', 'Más de 7 millones',
       'Entre 1 millón y menos de 2.5 millones',
       'Entre 500 mil y menos de 1 millón', 'Menos de 500 mil',
       'No pagó matrícula', nan], dtype=object)

In [40]:

mapping_matricula = {
    'No pagó matrícula': 0,
    'Menos de 500 mil': 1,
    'Entre 500 mil y menos de 1 millón': 2,
    'Entre 1 millón y menos de 2.5 millones': 3,
    'Entre 2.5 millones y menos de 4 millones': 4,
    'Entre 4 millones y menos de 5.5 millones': 5,
    'Entre 5.5 millones y menos de 7 millones': 6,
    'Más de 7 millones': 7
}

# nueva columna ordinal (Int64 para preservar NA)
df['E_VALORMATRICULAUNIVERSIDAD'] = df['E_VALORMATRICULAUNIVERSIDAD'].map(mapping_matricula).astype('Int64')


In [42]:
df["E_HORASSEMANATRABAJA"].unique()

array(['Menos de 10 horas', '0', 'Más de 30 horas', 'Entre 21 y 30 horas',
       'Entre 11 y 20 horas', nan], dtype=object)

In [43]:
mapping_horas = {
    '0': 0,
    'Menos de 10 horas': 1,
    'Entre 11 y 20 horas': 2,
    'Entre 21 y 30 horas': 3,
    'Más de 30 horas': 4
}

df['E_HORASSEMANATRABAJA'] = df['E_HORASSEMANATRABAJA'].map(mapping_horas).astype('Int64')

In [45]:
df["F_EDUCACIONPADRE"].unique()

array(['Técnica o tecnológica incompleta',
       'Técnica o tecnológica completa',
       'Secundaria (Bachillerato) completa', 'No sabe',
       'Primaria completa', 'Educación profesional completa',
       'Educación profesional incompleta', 'Primaria incompleta',
       'Postgrado', nan, 'Secundaria (Bachillerato) incompleta',
       'Ninguno', 'No Aplica'], dtype=object)

In [47]:
mapping_educacion = {
    'Ninguno': 0,
    'Primaria incompleta': 1,
    'Primaria completa': 2,
    'Secundaria (Bachillerato) incompleta': 3,
    'Secundaria (Bachillerato) completa': 4,
    'Técnica o tecnológica incompleta': 5,
    'Técnica o tecnológica completa': 6,
    'Educación profesional incompleta': 7,
    'Educación profesional completa': 8,
    'Postgrado': 9,
    # Estos se dejan como NA (no interpretables ordinalmente)
    'No sabe': pd.NA,
    'No Aplica': pd.NA
}

# 1) Columna ordinal numérica (Int64 para preservar NA)
df['F_EDUCACIONPADRE'] = df['F_EDUCACIONPADRE'].map(mapping_educacion).astype('Int64')

In [48]:
df['F_EDUCACIONMADRE'] = df['F_EDUCACIONMADRE'].map(mapping_educacion).astype('Int64')

In [49]:
df.drop(columns=['F_EDUCACIONPADRE_ORD'], inplace=True)

In [51]:
df["F_ESTRATOVIVIENDA"].unique()

array(['Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 2', 'Estrato 1',
       nan, 'Estrato 6', 'Sin Estrato'], dtype=object)

In [52]:
mapping_estrato = {
    'Estrato 1': 1,
    'Estrato 2': 2,
    'Estrato 3': 3,
    'Estrato 4': 4,
    'Estrato 5': 5,
    'Estrato 6': 6,
    "Sin estrato": pd.NA
}

# 1) Columna ordinal numérica (Int64 para preservar NA)
df['F_ESTRATOVIVIENDA'] = df['F_ESTRATOVIVIENDA'].map(mapping_estrato).astype('Int64')

In [53]:
df.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,RENDIMIENTO_GLOBAL_NUM,E_PRGM_ACADEMICO_ENC,E_PRGM_DEPARTAMENTO_ENC
0,20212,6,1,3,0,5,0,0,0,0,9,medio-alto,0.322,0.208,0.310,0.267,3,2.313951,2.557377
1,20212,4,0,3,1,6,0,1,0,0,5,bajo,0.311,0.215,0.292,0.264,1,2.629414,2.454803
2,20203,4,4,3,0,4,0,1,0,1,4,bajo,0.297,0.214,0.305,0.264,1,2.480084,2.557377
3,20195,5,0,4,0,<NA>,0,1,0,0,4,alto,0.485,0.172,0.252,0.190,4,2.298893,2.734127
4,20212,4,3,3,0,2,0,0,0,0,2,medio-bajo,0.316,0.232,0.285,0.294,2,2.466328,2.667561


In [54]:
df.shape

(692500, 19)


### Imputación de valores nulos y eliminación de filas

En las siguientes celdas se realiza el tratamiento de valores nulos en el dataset `df` siguiendo las reglas descritas:

1. **Eliminación de filas con más del 50% de valores nulos**:
    - Se calcula el umbral de columnas nulas permitido por fila (50% del total de columnas).
    - Si una fila tiene más valores nulos que este umbral, se elimina del dataset.

2. **Imputación de valores nulos en las filas restantes**:
    - Para variables categóricas transformadas a ordinales o binarias:
      - Se rellenan los valores nulos con la moda (valor más frecuente) de la columna.
    - Para variables numéricas:
      - Se rellenan los valores nulos con la media de la columna.

Este proceso asegura que el dataset quede limpio y sin valores nulos, manteniendo la integridad de los datos y preparándolos para su uso en modelos de Machine Learning.

In [55]:

# 1) Normalizar placeholders de "desconocido" a pd.NA (ajusta la lista si tienes otros valores)
unknown_strings = [
    'No sabe', 'No Aplica', 'Desconocido', 'Desconocida',
    'Unknown', 'NA', 'N/A', 'Sin dato', 'sin dato', ''
]
# Reemplazamos (inplace)
df.replace(unknown_strings, pd.NA, inplace=True)

In [56]:
# 2) Eliminar filas con más del 50% de columnas nulas
n_cols = df.shape[1]
threshold = 0.5 * n_cols

# contar NaNs por fila (usando isna() que reconoce pd.NA y np.nan)
null_counts = df.isna().sum(axis=1)
rows_to_drop = df.index[null_counts > threshold]

print(f"Filas totales antes: {len(df)}")
print(f"Filas a eliminar (>{int(threshold)} columnas nulas): {len(rows_to_drop)}")

Filas totales antes: 692500
Filas a eliminar (>9 columnas nulas): 1639


In [57]:
# eliminar inplace
if len(rows_to_drop) > 0:
    df.drop(index=rows_to_drop, inplace=True)
    df.reset_index(drop=True, inplace=True)
    print(f"Filas totales después de eliminar: {len(df)}")
else:
    print("No se eliminaron filas (ninguna supera el 50% de nulos).")

Filas totales después de eliminar: 690861


In [58]:
from pandas.api.types import is_categorical_dtype, is_numeric_dtype, is_integer_dtype

In [59]:
fill_values = {}  # guardará la estrategia y el valor usado por columna

# detectar si existe binary_maps (lo creaste antes); si no, lo ignoramos
binary_map_keys = set(globals().get('binary_maps', {}).keys()) if 'binary_maps' in globals() else set()

for col in df.columns:
    # recuento de nulos actuales
    n_null = df[col].isna().sum()
    if n_null == 0:
        continue  # nada que imputar

    col_info = {}
    # criterio para considerar "categorica ordinal/binaria":
    # - dtype category OR
    # - sufijo _ORD o _CATORD OR
    # - columna presente en binary_maps
    is_cat_ord = (
        is_categorical_dtype(df[col]) or
        str(col).endswith('_ORD') or
        str(col).endswith('_CATORD') or
        (col in binary_map_keys)
    )

    if is_cat_ord:
        # usar moda (valor más frecuente)
        try:
            mode_series = df[col].mode(dropna=True)
            if len(mode_series) > 0:
                impute_val = mode_series.iloc[0]
            else:
                # si no hay moda (p. ej. todos NaN) -> dejar como NA (no imputamos)
                impute_val = pd.NA
        except Exception:
            impute_val = pd.NA

        # asignar inplace manteniendo dtype; para Int64 hacemos cast si es necesario
        if pd.isna(impute_val):
            # no hay valor para imputar -> saltar
            col_info['strategy'] = 'mode'
            col_info['value'] = None
            fill_values[col] = col_info
            continue

        # Si la columna es Int64 o tenía ese tipo, convertir a Int64 tras fill
        if df[col].dtype.name == 'Int64' or is_integer_dtype(df[col].dtype):
            # impute_val puede ser numpy int, python int, or category
            try:
                df[col].fillna(int(impute_val), inplace=True)
                df[col] = df[col].astype('Int64')
                col_info['strategy'] = 'mode'
                col_info['value'] = int(impute_val)
            except Exception:
                # en caso de fallo (p. ej. impute_val no convertible), usar raw fill
                df[col].fillna(impute_val, inplace=True)
                col_info['strategy'] = 'mode'
                col_info['value'] = impute_val
        else:
            df[col].fillna(impute_val, inplace=True)
            col_info['strategy'] = 'mode'
            col_info['value'] = impute_val

    else:
        # tratar como numérica: imputar con la media
        # si no es numérica, intentaremos convertirla; si falla, usaremos la moda como fallback
        if is_numeric_dtype(df[col]):
            mean_val = df[col].mean(skipna=True)
            if pd.isna(mean_val):
                # no hay datos numéricos -> fallback a moda
                mode_series = df[col].mode(dropna=True)
                impute_val = mode_series.iloc[0] if len(mode_series) > 0 else pd.NA
                df[col].fillna(impute_val, inplace=True)
                col_info['strategy'] = 'mode_fallback'
                col_info['value'] = impute_val
            else:
                # si columna era entero nullable, redondeamos la media
                if df[col].dtype.name == 'Int64' or is_integer_dtype(df[col].dtype):
                    impute_val = int(round(mean_val))
                    df[col].fillna(impute_val, inplace=True)
                    df[col] = df[col].astype('Int64')
                    col_info['strategy'] = 'mean_rounded'
                    col_info['value'] = impute_val
                else:
                    df[col].fillna(mean_val, inplace=True)
                    col_info['strategy'] = 'mean'
                    col_info['value'] = float(mean_val)
        else:
            # no numérica; intentar convertir a numérico
            coerced = pd.to_numeric(df[col], errors='coerce')
            if coerced.notna().sum() > 0:
                mean_val = coerced.mean(skipna=True)
                impute_val = mean_val
                # rellenar en la columna original (dejando tipo original) con el valor numérico
                df[col] = df[col].fillna(impute_val)
                col_info['strategy'] = 'mean_coerced'
                col_info['value'] = float(impute_val)
            else:
                # fallback: usar la moda
                mode_series = df[col].mode(dropna=True)
                impute_val = mode_series.iloc[0] if len(mode_series) > 0 else pd.NA
                df[col].fillna(impute_val, inplace=True)
                col_info['strategy'] = 'mode_fallback_non_numeric'
                col_info['value'] = impute_val

    fill_values[col] = col_info

# 4) Resumen final
print("\nResumen de imputaciones (columna: estrategia -> valor):")
for c, info in fill_values.items():
    print(f" - {c}: {info['strategy']} -> {info['value']}")

print("\nComprobación rápida de nulos residuales por columna (debería ser 0 en la mayoría):")
print(df.isna().sum().sort_values(ascending=False).head(20))



Resumen de imputaciones (columna: estrategia -> valor):
 - E_VALORMATRICULAUNIVERSIDAD: mean_rounded -> 4
 - E_HORASSEMANATRABAJA: mean_rounded -> 2
 - F_ESTRATOVIVIENDA: mean_rounded -> 3
 - F_TIENEINTERNET: mode -> 0
 - F_EDUCACIONPADRE: mean_rounded -> 4
 - F_TIENELAVADORA: mode -> 0
 - F_TIENEAUTOMOVIL: mode -> 1
 - E_PAGOMATRICULAPROPIO: mode -> 0
 - F_TIENECOMPUTADOR: mode -> 0
 - F_EDUCACIONMADRE: mean_rounded -> 4

Comprobación rápida de nulos residuales por columna (debería ser 0 en la mayoría):
PERIODO_ACADEMICO              0
E_VALORMATRICULAUNIVERSIDAD    0
E_HORASSEMANATRABAJA           0
F_ESTRATOVIVIENDA              0
F_TIENEINTERNET                0
F_EDUCACIONPADRE               0
F_TIENELAVADORA                0
F_TIENEAUTOMOVIL               0
E_PAGOMATRICULAPROPIO          0
F_TIENECOMPUTADOR              0
F_EDUCACIONMADRE               0
RENDIMIENTO_GLOBAL             0
INDICADOR_1                    0
INDICADOR_2                    0
INDICADOR_3                

C:\Users\JUANJO\AppData\Local\Temp\ipykernel_12244\2626026109.py:18: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  is_categorical_dtype(df[col]) or
C:\Users\JUANJO\AppData\Local\Temp\ipykernel_12244\2626026109.py:78: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(impute_val, inplace=True)
C:\Users\JUANJO\AppData\Local\Temp\ipykernel_12244\2626026109.py:18: DeprecationWarning: is_categorical_dtype is deprecated a

In [61]:
df.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,RENDIMIENTO_GLOBAL_NUM,E_PRGM_ACADEMICO_ENC,E_PRGM_DEPARTAMENTO_ENC
0,20212,6,1,3,0,5,0,0,0,0,9,medio-alto,0.322,0.208,0.310,0.267,3,2.313951,2.557377
1,20212,4,0,3,1,6,0,1,0,0,5,bajo,0.311,0.215,0.292,0.264,1,2.629414,2.454803
2,20203,4,4,3,0,4,0,1,0,1,4,bajo,0.297,0.214,0.305,0.264,1,2.480084,2.557377
3,20195,5,0,4,0,4,0,1,0,0,4,alto,0.485,0.172,0.252,0.190,4,2.298893,2.734127
4,20212,4,3,3,0,2,0,0,0,0,2,medio-bajo,0.316,0.232,0.285,0.294,2,2.466328,2.667561


In [ ]:
#El número total de valores nulos en el DataFrame después del preprocesamiento
df.isna().sum().sum()

np.int64(0)


### Normalización de Variables Numéricas

En las celdas correspondientes, se realiza la normalización de las variables numéricas seleccionadas en el DataFrame `df`. Este proceso asegura que todas las variables estén en la misma escala, lo cual es importante para algoritmos de Machine Learning sensibles a las magnitudes de las variables. A continuación, se describe el procedimiento:

1. **Selección de columnas a normalizar**:
    - Se define una lista `cols_to_scale` con los nombres de las columnas que se desean normalizar.
    - Se verifica la existencia de estas columnas en el DataFrame `df`. Si alguna columna no está presente, se omite y se emite una advertencia.

2. **Conversión a tipo `float`**:
    - Las columnas seleccionadas se convierten al tipo `float` para evitar problemas con valores nulos (`NA`) o tipos de datos como `Int64`.

3. **Cálculo de parámetros de normalización**:
    - Para cada columna, se calcula el valor mínimo (`mins`) y el valor máximo (`maxs`).
    - Se calcula el denominador como la diferencia entre el máximo y el mínimo (`maxs - mins`). Si esta diferencia es 0 (cuando todos los valores son iguales), se reemplaza por 1 para evitar divisiones por cero.

4. **Aplicación de la normalización**:
    - Se utiliza la fórmula de normalización min-max:
      \[
      X_{\text{normalizado}} = \frac{X - \text{min}}{\text{max} - \text{min}}
      \]
    - Esto transforma los valores de cada columna en un rango de 0 a 1.

5. **Almacenamiento de parámetros de escala**:
    - Los valores mínimos y máximos utilizados para cada columna se guardan en un diccionario `scale_params` para referencia futura.

6. **Resumen post-normalización**:
    - Se imprime un resumen de las columnas normalizadas, junto con los valores mínimos y máximos utilizados.
    - También se verifica que los valores normalizados estén efectivamente en el rango [0, 1].

Este enfoque asegura que las variables numéricas sean escaladas de manera consistente, preservando las relaciones relativas entre los valores.


In [66]:
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype("Int64")

In [70]:
cols_to_scale = [
    'PERIODO_ACADEMICO',
    'INDICADOR_1',
    'INDICADOR_2',
    'INDICADOR_3',
    'INDICADOR_4',
    'E_PRGM_ACADEMICO_ENC',
    'E_PRGM_DEPARTAMENTO_ENC'
]

# comprobar existencia de columnas
present = [c for c in cols_to_scale if c in df.columns]
missing = [c for c in cols_to_scale if c not in df.columns]
if missing:
    print("Advertencia: las siguientes columnas no existen en df y serán omitidas:", missing)

if len(present) == 0:
    print("No hay columnas válidas para normalizar. Revisa los nombres.")
else:
    # convertir a float para evitar problemas con Int64/NA al calcular min/max
    arr = df[present].astype(float)

    mins = arr.min(skipna=True)
    maxs = arr.max(skipna=True)
    denom = (maxs - mins).replace(0, 1)  # evitar división por cero cuando max==min

    df[present] = (arr - mins) / denom

    # guardar parámetros de escala por si los necesitas luego
    scale_params = {c: (float(mins[c]), float(maxs[c])) for c in present}

    print("Columnas normalizadas (0-1):", present)
    print("\nValores min/max usados:")
    for c in present:
        print(f" - {c}: min={scale_params[c][0]}, max={scale_params[c][1]}")

    # resumen rápido post-normalización
    print("\nResumen post-normalización (min, max) por columna (NaNs ignorados):")
    for c in present:
        mn = df[c].min(skipna=True)
        mx = df[c].max(skipna=True)
        print(f" - {c}: min={mn:.6f}, max={mx:.6f}")

Columnas normalizadas (0-1): ['PERIODO_ACADEMICO', 'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4', 'E_PRGM_ACADEMICO_ENC', 'E_PRGM_DEPARTAMENTO_ENC']

Valores min/max usados:
 - PERIODO_ACADEMICO: min=20183.0, max=20213.0
 - INDICADOR_1: min=0.0, max=0.657
 - INDICADOR_2: min=0.0, max=0.487
 - INDICADOR_3: min=0.0, max=0.32
 - INDICADOR_4: min=0.0, max=0.332
 - E_PRGM_ACADEMICO_ENC: min=1.3466317257318603, max=3.745709920361281
 - E_PRGM_DEPARTAMENTO_ENC: min=1.369715606576111, max=2.930125698015135

Resumen post-normalización (min, max) por columna (NaNs ignorados):
 - PERIODO_ACADEMICO: min=0.000000, max=1.000000
 - INDICADOR_1: min=0.000000, max=1.000000
 - INDICADOR_2: min=0.000000, max=1.000000
 - INDICADOR_3: min=0.000000, max=1.000000
 - INDICADOR_4: min=0.000000, max=1.000000
 - E_PRGM_ACADEMICO_ENC: min=0.000000, max=1.000000
 - E_PRGM_DEPARTAMENTO_ENC: min=0.000000, max=1.000000



### Explicación de las últimas celdas del archivo

En las últimas celdas del archivo, se realizan las siguientes operaciones:

1. **Separación de la variable objetivo (`y`)**:
    - En la celda 44, se extrae la columna `RENDIMIENTO_GLOBAL` del DataFrame `df` y se asigna a la variable `y`. Esta columna representa la variable objetivo del modelo.
    - Posteriormente, se eliminan las columnas `RENDIMIENTO_GLOBAL` y `RENDIMIENTO_GLOBAL_NUM` del DataFrame `df`, dejando únicamente las características predictoras.

2. **Codificación One-Hot de la variable objetivo (`y`)**:
    - En la celda 45, se utiliza la clase `OneHotEncoder` de `sklearn` para realizar una codificación One-Hot de la variable `y`. Esto transforma las categorías de `RENDIMIENTO_GLOBAL` en un formato binario, donde cada categoría se representa como una columna con valores 0 o 1.
    - El resultado se almacena en la variable `y_one_hot`, que es una matriz dispersa (`sparse matrix`) para optimizar el uso de memoria.

3. **Conversión de la matriz dispersa a un DataFrame**:
    - En la celda 46, la matriz dispersa `y_one_hot` se convierte en un DataFrame utilizando `pd.DataFrame.sparse.from_spmatrix`. Las columnas del DataFrame resultante se nombran utilizando los nombres de las categorías generados por `OneHotEncoder`.

4. **Visualización de los datos**:
    - En la celda 47, se utiliza `df.head()` para mostrar las primeras filas del DataFrame `df`, permitiendo verificar el estado de las características después del preprocesamiento.

5. **Copia del DataFrame de características (`X`)**:
    - En la celda 48, se realiza una copia del DataFrame `df` y se asigna a la variable `X`. Esto asegura que `X` contenga únicamente las características predictoras, separadas de la variable objetivo.

6. **Guardado de los conjuntos preprocesados**:
    - En la celda 49, se guardan los conjuntos preprocesados `X` y `y` en archivos CSV (`X_preprocessed.csv` y `y_preprocessed.csv`) para su uso posterior.
    - En la celda 50, se guarda el DataFrame `y_df` (resultado de la codificación One-Hot) en un archivo CSV (`y_one_hot_preprocessed.csv`).

Estas celdas finalizan el preprocesamiento de los datos, dejando los conjuntos de características (`X`) y la variable objetivo (`y`) listos para ser utilizados en el entrenamiento de modelos de Machine Learning.


In [64]:
y = df['RENDIMIENTO_GLOBAL']
df.drop(columns=['RENDIMIENTO_GLOBAL', 'RENDIMIENTO_GLOBAL_NUM'], inplace=True)

In [76]:
# One hot encoding del target con sklearn
from sklearn.preprocessing import OneHotEncoder

y_one_hot = OneHotEncoder().fit_transform(y.values.reshape(-1, 1))

In [78]:
y_df = pd.DataFrame.sparse.from_spmatrix(y_one_hot, columns=OneHotEncoder().fit(y.values.reshape(-1, 1)).get_feature_names_out(['RENDIMIENTO_GLOBAL']))

In [72]:
df.head()

,PERIODO_ACADEMICO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,F_TIENEAUTOMOVIL,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_EDUCACIONMADRE,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4,E_PRGM_ACADEMICO_ENC,E_PRGM_DEPARTAMENTO_ENC
0,0.966667,6,1,3,0,5,0,0,0,0,9,0.490107,0.427105,0.968750,0.804217,0.403205,0.761121
1,0.966667,4,0,3,1,6,0,1,0,0,5,0.473364,0.441478,0.912500,0.795181,0.534698,0.695386
2,0.666667,4,4,3,0,4,0,1,0,1,4,0.452055,0.439425,0.953125,0.795181,0.472453,0.761121
3,0.400000,5,0,4,0,4,0,1,0,0,4,0.738204,0.353183,0.787500,0.572289,0.396928,0.874393
4,0.966667,4,3,3,0,2,0,0,0,0,2,0.480974,0.476386,0.890625,0.885542,0.466719,0.831733


In [73]:
X = df.copy()

In [ ]:
# Guardar los conjuntos preprocesados
X.to_csv('data/X_preprocessed.csv', index=False)
y.to_csv('data/y_preprocessed.csv', index=False)

In [80]:
y_df.to_csv('data/y_one_hot_preprocessed.csv', index=False)

### Conclusión del Preprocesamiento de Datos

El preprocesamiento realizado en este archivo ha permitido transformar y preparar un conjunto de datos extenso y complejo para su uso en modelos de Machine Learning. A continuación, se resumen los puntos clave:

1. **Limpieza y Transformación de Datos**:
    - Se eliminaron columnas redundantes o irrelevantes, como `ID`, `E_PRIVADO_LIBERTAD`, y `F_TIENEINTERNET.1`.
    - Se imputaron valores nulos utilizando estrategias específicas para cada tipo de variable (moda para categóricas y media para numéricas).
    - Se normalizaron las variables numéricas seleccionadas, asegurando que todas estén en la misma escala (rango [0, 1]).

2. **Codificación de Variables Categóricas**:
    - Se aplicaron técnicas de codificación como el encoding ordinal y el encoding suavizado para variables categóricas, como `E_PRGM_ACADEMICO` y `E_PRGM_DEPARTAMENTO`.
    - Las variables binarias fueron transformadas a valores numéricos (0 y 1), preservando valores nulos cuando fue necesario.
    - La variable objetivo `RENDIMIENTO_GLOBAL` fue transformada a una representación ordinal y también se realizó una codificación One-Hot para su uso en diferentes modelos.

3. **Análisis de Independencia y Relación**:
    - Se realizaron pruebas de Chi-cuadrado para evaluar la independencia entre variables categóricas y la variable objetivo, identificando relaciones significativas.

4. **Preparación de Conjuntos de Datos**:
    - Se separaron las características predictoras (`X`) y la variable objetivo (`y`), asegurando que los datos estén listos para el entrenamiento de modelos.
    - Los conjuntos preprocesados fueron guardados en archivos CSV para su uso posterior.

5. **Escalabilidad y Eficiencia**:
    - El preprocesamiento fue diseñado para manejar un conjunto de datos grande (690,861 registros), optimizando el uso de memoria y preservando la integridad de los datos.

En general, este archivo demuestra un flujo de trabajo robusto y bien documentado para el preprocesamiento de datos, asegurando que el conjunto de datos esté limpio, transformado y listo para ser utilizado en tareas de modelado predictivo. Este enfoque garantiza que los modelos puedan aprovechar al máximo la información contenida en los datos, minimizando el impacto de valores atípicos, nulos o inconsistencias.